In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
NY_reviews = pd.read_csv('/content/drive/MyDrive/DS4A Team4/New_York_reviews_cleaned.csv', index_col=0)
NY_reviews.head()

,restaurant_name,rating_review,sample,title_review,review_preview,review_full,date,author_id
1,Lido,5,Positive,a regular treat,my wife and i have been eating dinner frequent...,my wife and i have been eating dinner frequent...,2020-10-08,UID_0
2,Lido,4,Positive,good neighborhood spot!,came with family for labor day weekend brunch ...,came with family for labor day weekend brunch ...,2020-09-08,UID_1
3,Lido,1,Negative,disappointing,food was mediocre at best. the lamb chops are...,food was mediocre at best. the lamb chops are ...,2020-02-17,UID_2
4,Lido,5,Positive,what a find in harlem,my co-workers were volunteering at a foodbank ...,my co-workers were volunteering at a foodbank ...,2019-11-25,UID_3
5,Lido,5,Positive,lunch,lido is an intimate boutique style restaurant....,lido is an intimate boutique style restaurant....,2019-11-23,UID_4


In [4]:
grouped = NY_reviews[['restaurant_name','review_full']].groupby('restaurant_name', as_index=False).agg(' '. join)

In [11]:
query = NY_reviews[['author_id','review_full']].groupby('author_id', as_index=False).agg(' '. join)

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from collections import  Counter
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

STOPWORDS = set(stopwords.words('english'))
MIN_WORDS = 4
MAX_WORDS = 200

PATTERN_S = re.compile("\'s")  # matches `'s` from text
PATTERN_RN = re.compile("\\r\\n") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace

def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers.
        text (str): input text
    return (str): modified initial text
    """
    text = text.lower()  # lowercase text
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

def tokenizer(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return token


def clean_sentences(df):
    """
    Remove irrelavant characters (in new column clean_sentence).
    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).
    """
    print('Cleaning sentences...')
    df['clean_sentence'] = df['review_full'].apply(clean_text)
    df['tok_lem_sentence'] = df['clean_sentence'].apply(
        lambda x: tokenizer(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True))
    return df

df = clean_sentences(NY_reviews)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Cleaning sentences...


In [12]:
df = grouped
df.columns = ['restaurant_names','sentence']
query.columns = ['author_id','sentence']

## Bert

In [7]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 611.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=862bb00e1d97502fc195a8154c3009358b57af8c9917d77cbca16149f990f245
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [13]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
corpus_embeddings = model.encode(df.sentence.values, convert_to_tensor=True)
query_embedding = model.encode(query.sentence.values, convert_to_tensor=True)

In [22]:
import torch

# We use cosine-similarity and torch.topk to find the highest 3 scores
cos_scores = util.pytorch_cos_sim(query_embedding[0], corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=3)

print("\n\n======================\n\n")
print("Query:", query.author_id[0])
print("\nTop 5 most similar sentences in corpus:")

for score, idx in zip(top_results[0], top_results[1]):
    score = score.cpu().data.numpy()
    idx = idx.cpu().data.numpy()
    display(df[['restaurant_names']].iloc[idx])





Query: UID_0

Top 5 most similar sentences in corpus:


restaurant_names    Lido
Name: 920, dtype: object

restaurant_names    44_X
Name: 11, dtype: object

restaurant_names    The_East_Pole
Name: 1572, dtype: object

In [37]:
import torch

cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)

In [45]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-mpnet-base-v2')
corpus_embeddings = model.encode(df.sentence.values, convert_to_tensor=True)
query_embedding = model.encode(query.sentence.values, convert_to_tensor=True)

In [46]:
corpus_embeddings.size()

torch.Size([1805, 768])

In [48]:
query_embedding.size()

torch.Size([249901, 768])

In [55]:
import torch

# We use cosine-similarity and torch.topk to find the highest 3 scores
cos_scores = util.pytorch_cos_sim(query_embedding[0], corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=3)

print("\n\n======================\n\n")
print("Query:", query.author_id[0])
print("\nTop 5 most similar sentences in corpus:")

for score, idx in zip(top_results[0], top_results[1]):
    score = score.cpu().data.numpy()
    idx = idx.cpu().data.numpy()
    display(df[['restaurant_names']].iloc[idx])





Query: UID_0

Top 5 most similar sentences in corpus:


restaurant_names    Lido
Name: 920, dtype: object

restaurant_names    44_X
Name: 11, dtype: object

restaurant_names    John_s_of_12th_Street
Name: 806, dtype: object

In [56]:
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)

In [57]:
cos_scores.size()

torch.Size([249901, 1805])